In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

df = pd.read_csv("./data-source/data.csv")
df.head(5)

,location,latitude,longitude,date,time,temperature,uv_index,wind_speed,cloud_cover,precip_intensity
0,Camden East,44.335385,-76.833312,2017-01-01,9:00:00,-1.52,0,7.77,0.75,0
1,Camden East,44.335385,-76.833312,2017-01-01,10:00:00,-1.13,1,7,0.75,0
2,Camden East,44.335385,-76.833312,2017-01-01,11:00:00,-0.32,1,7,0.75,0
3,Camden East,44.335385,-76.833312,2017-01-01,12:00:00,-0.12,1,4.67,1,0
4,Camden East,44.335385,-76.833312,2017-01-01,13:00:00,0.88,1,8.69,1,0


PART 1 - CLEAN DATA

In [2]:
#examine datatypes
df.dtypes

location             object
latitude            float64
longitude           float64
date                 object
time                 object
temperature          object
uv_index             object
wind_speed           object
cloud_cover          object
precip_intensity     object
dtype: object

In [3]:
#convert data types 
cols = ['temperature','uv_index', 'wind_speed', 'cloud_cover', 'precip_intensity']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df['date'] = df['date'].astype('datetime64[ns]')
#df.set_index('date',inplace=True)

In [4]:
#derive two new columns
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [5]:
df.dtypes

location                    object
latitude                   float64
longitude                  float64
date                datetime64[ns]
time                        object
temperature                float64
uv_index                   float64
wind_speed                 float64
cloud_cover                float64
precip_intensity           float64
month                        int64
year                         int64
dtype: object

In [6]:
#how abt % of missing values per column
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing

location             0.000000
latitude             0.000000
longitude            0.000000
date                 0.000000
time                 0.000000
temperature          0.283342
uv_index            18.123649
wind_speed           0.639008
cloud_cover         18.489526
precip_intensity    16.611643
month                0.000000
year                 0.000000
dtype: float64

In [7]:
#NaN stats
df.isnull().sum()

location                0
latitude                0
longitude               0
date                    0
time                    0
temperature           333
uv_index            21300
wind_speed            751
cloud_cover         21730
precip_intensity    19523
month                   0
year                    0
dtype: int64

In [8]:
#fill missing NaNs using ffill followed by bfill. 
#ffill to replace by taking most recent non-NaN value before within the same date
#bfill in case first hour of day is missing (i.e 9 am) in that case take the most recent valid value after but within the same date

missing_vals = ['temperature','wind_speed','uv_index','cloud_cover','precip_intensity']

for i in missing_vals:
    df[i] = df.groupby(['location','date'])[i].ffill()
    df[i] = df.groupby(['location','date'])[i].bfill()

    

In [9]:
#recheck Nan stats, some reduction
df.isnull().sum()

location                0
latitude                0
longitude               0
date                    0
time                    0
temperature             0
uv_index            18247
wind_speed            450
cloud_cover         21234
precip_intensity    18532
month                   0
year                    0
dtype: int64

In [10]:
#replace remaining Nans for WHOLE DAY with randomly selected value specific to that month, year

#get NaN indices
cols = ['wind_speed','uv_index','cloud_cover','precip_intensity']
nan_index = []

for i in cols:
    nan_index.append(df[i].index[df[i].apply(np.isnan)].tolist())
    
i = -1

for c in cols:
    i += 1
    for j in nan_index[i]:
        month_period = df.loc[df.index == j,'month'].item()
        year_period = df.loc[df.index == j,'year'].item()
        df[c] = np.where(df.index == j
                         ,df.loc[(df.month == month_period) & (df.year == year_period)][c].dropna().sample(replace=True).item()
                         ,df[c])
        
        



#Option 1 - fill remaining Nans with mean for that month, year
#df['wind_speed'] = df.groupby(['location', df.date.dt.year,df.date.dt.month])['wind_speed'].transform(lambda x: x.fillna(x.mean()))



In [11]:
#recheck Nans
df.isnull().sum()

location            0
latitude            0
longitude           0
date                0
time                0
temperature         0
uv_index            0
wind_speed          0
cloud_cover         0
precip_intensity    0
month               0
year                0
dtype: int64

In [12]:
#group data by avg

dg =df.groupby(['location',df.date.dt.year,df.date.dt.month],as_index=False).mean()

dg.head(20)

,location,latitude,longitude,temperature,uv_index,wind_speed,cloud_cover,precip_intensity,month,year
0,Camden East,44.335385,-76.833312,-1.745878,0.645161,12.099462,0.803369,0.034537,1,2017
1,Camden East,44.335385,-76.833312,-0.831468,1.317460,12.770714,0.776032,0.042927,2,2017
2,Camden East,44.335385,-76.833312,-1.411075,2.397849,14.337885,0.731756,0.069929,3,2017
3,Camden East,44.335385,-76.833312,9.895111,3.551852,13.354222,0.753333,0.169060,4,2017
4,Camden East,44.335385,-76.833312,13.821756,4.000000,11.579247,0.815986,0.213444,5,2017
5,Camden East,44.335385,-76.833312,19.797889,4.640741,12.168704,0.706000,0.142989,6,2017
6,Camden East,44.335385,-76.833312,22.409355,4.476703,10.311828,0.692007,0.127911,7,2017
7,Camden East,44.335385,-76.833312,21.786846,3.956989,10.368746,0.656918,0.050216,8,2017
8,Camden East,44.335385,-76.833312,20.453519,2.988889,8.847667,0.543704,0.037614,9,2017
9,Camden East,44.335385,-76.833312,14.774982,1.663082,13.917634,0.689570,0.195141,10,2017


PART 2 - MERGE PV POTENTIALS WITH CLEANED DATA

In [13]:
dp = pd.read_csv("./data-source/municip_potentiel-potential.csv", encoding = "ISO-8859-1")
dp.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,PV potential / Potentiel PV (kWh/kWp),PV potential / Potentiel PV (kWh/kWp).1,PV potential / Potentiel PV (kWh/kWp).2,PV potential / Potentiel PV (kWh/kWp).3
0,Province,Municipality,month,South-facing vertical (tilt=90°),South-facing tilt=latitude,South-facing tilt=lat+15°,South-facing tilt=lat-15°
1,province,municipalité,mois,vertical orienté vers le sud (inc.=90°),orienté vers le sud (inc.=latitude),orienté vers le sud (inc.=latitude +15°),orienté vers le sud (inc.=latitude -15°)
2,Nova Scotia/Nouvelle-Écosse,St. Peter's,January,56,55,58,50
3,Nova Scotia/Nouvelle-Écosse,St. Peter's,February,71,75,77,69
4,Nova Scotia/Nouvelle-Écosse,St. Peter's,March,83,102,99,99


In [14]:
#get Ontario  data only
dp = dp[dp['Unnamed: 0']=='Ontario'].reset_index(drop=True)

In [15]:
dp = dp.rename(index=str, columns={"Unnamed: 0": "Province", "Unnamed: 1": "location", "Unnamed: 2": "month",\
                                  "PV potential / Potentiel PV (kWh/kWp)": "PVpot tilt=90",\
                                  "PV potential / Potentiel PV (kWh/kWp).1": "PVpot tilt=lat",\
                                  "PV potential / Potentiel PV (kWh/kWp).2": "PVpot tilt=lat+15",\
                                  "PV potential / Potentiel PV (kWh/kWp).3": "PVpot tilt=lat-15"})

#drop Annual based rows. 
dp = dp[dp.month != 'Annual']

dp.dtypes

Province             object
location             object
month                object
PVpot tilt=90        object
PVpot tilt=lat       object
PVpot tilt=lat+15    object
PVpot tilt=lat-15    object
dtype: object

In [16]:
#change data type of PV potential colums to float
columns = ['PVpot tilt=90','PVpot tilt=lat','PVpot tilt=lat+15','PVpot tilt=lat-15']
dp[columns] = dp[columns].apply(pd.to_numeric, downcast = 'float',errors='coerce')

dp.dtypes

Province              object
location              object
month                 object
PVpot tilt=90        float32
PVpot tilt=lat       float32
PVpot tilt=lat+15    float32
PVpot tilt=lat-15    float32
dtype: object

In [17]:
#derive new columns
dp['AvgPVPotential'] = (dp['PVpot tilt=90'] + dp['PVpot tilt=lat'] + dp['PVpot tilt=lat+15'] + dp['PVpot tilt=lat-15'])/4

#PVPotential to be lat+15 for Jan-Mar,Oct-Dec, otherwise lat-15 as per optimal tilts
def get_PVPotential(row):
    latplus15_months = ['January','February','March','October','November','December']
    if row['month'] in latplus15_months:
        return row['PVpot tilt=lat+15']
    else:
        return row['PVpot tilt=lat-15']
    
dp['PVPotential'] = dp.apply (lambda row: get_PVPotential(row), axis=1)


In [18]:
dp.head(20)

,Province,location,month,PVpot tilt=90,PVpot tilt=lat,PVpot tilt=lat+15,PVpot tilt=lat-15,AvgPVPotential,PVPotential
0,Ontario,Quinte West,January,75.0,73.0,78.0,64.0,72.50,78.0
1,Ontario,Quinte West,February,84.0,88.0,91.0,81.0,86.00,91.0
2,Ontario,Quinte West,March,92.0,115.0,113.0,111.0,107.75,113.0
3,Ontario,Quinte West,April,70.0,113.0,103.0,116.0,100.50,116.0
4,Ontario,Quinte West,May,65.0,123.0,108.0,133.0,107.25,133.0
5,Ontario,Quinte West,June,60.0,123.0,105.0,135.0,105.75,135.0
6,Ontario,Quinte West,July,65.0,131.0,113.0,144.0,113.25,144.0
7,Ontario,Quinte West,August,70.0,121.0,109.0,128.0,107.00,128.0
8,Ontario,Quinte West,September,72.0,102.0,97.0,102.0,93.25,102.0
9,Ontario,Quinte West,October,75.0,89.0,89.0,84.0,84.25,89.0


In [19]:
#convert month int to month names
import calendar

dg['month'] = dg['month'].apply(lambda x: calendar.month_name[x])


In [20]:
dg.head(10)

,location,latitude,longitude,temperature,uv_index,wind_speed,cloud_cover,precip_intensity,month,year
0,Camden East,44.335385,-76.833312,-1.745878,0.645161,12.099462,0.803369,0.034537,January,2017
1,Camden East,44.335385,-76.833312,-0.831468,1.317460,12.770714,0.776032,0.042927,February,2017
2,Camden East,44.335385,-76.833312,-1.411075,2.397849,14.337885,0.731756,0.069929,March,2017
3,Camden East,44.335385,-76.833312,9.895111,3.551852,13.354222,0.753333,0.169060,April,2017
4,Camden East,44.335385,-76.833312,13.821756,4.000000,11.579247,0.815986,0.213444,May,2017
5,Camden East,44.335385,-76.833312,19.797889,4.640741,12.168704,0.706000,0.142989,June,2017
6,Camden East,44.335385,-76.833312,22.409355,4.476703,10.311828,0.692007,0.127911,July,2017
7,Camden East,44.335385,-76.833312,21.786846,3.956989,10.368746,0.656918,0.050216,August,2017
8,Camden East,44.335385,-76.833312,20.453519,2.988889,8.847667,0.543704,0.037614,September,2017
9,Camden East,44.335385,-76.833312,14.774982,1.663082,13.917634,0.689570,0.195141,October,2017


In [21]:
cleaned = pd.merge(dp, dg,  how='inner', on=['location', 'month'])

#can add more columns, if required...
cols = ['temperature','uv_index', 'wind_speed', 'cloud_cover','precip_intensity','PVPotential']
cleaned = cleaned[cols]


In [22]:
#print final cleaned data to use in regression, ML etc

cleaned.head(10)

,temperature,uv_index,wind_speed,cloud_cover,precip_intensity,PVPotential
0,-3.783835,0.602151,10.693978,0.772294,0.018808,84.0
1,-8.174803,0.616487,11.618423,0.805591,0.044539,84.0
2,-3.085040,1.277778,11.819484,0.736667,0.032070,101.0
3,-2.404643,1.246032,11.607937,0.734405,0.055358,101.0
4,-3.388925,2.415771,14.013226,0.646953,0.022958,123.0
5,0.121720,2.408602,12.841720,0.681541,0.040613,123.0
6,9.884741,3.503704,14.127111,0.725593,0.054833,116.0
7,5.224741,3.488889,14.517370,0.746963,0.083654,116.0
8,14.774552,4.086022,13.390358,0.771613,0.054051,129.0
9,18.657312,4.713262,12.589749,0.557814,0.054665,129.0
